# Fake face detector

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets, models
import torchvision.utils as vutils
import torch.nn.init as init

import matplotlib.pyplot as plt
import numpy as np
import os

from google.colab import files
import zipfile
import io

## Importación y preparación de datos

In [ ]:
uploaded = files.upload()

nombre_del_archivo = "real_and_fake_face.zip"

with zipfile.ZipFile(io.BytesIO(uploaded[nombre_del_archivo]), 'r') as zip_ref:
    zip_ref.extractall()

carpeta_de_imagenes = "real_and_fake_face"
lista_de_archivos = os.listdir(carpeta_de_imagenes)

print("Archivos en la carpeta de imágenes:")
print(lista_de_archivos)

In [ ]:
DATASET = os.path.join('real_and_fake_face')

In [ ]:
INITIAL_SIZE_OF_IMAGES = (128, 128)

data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(INITIAL_SIZE_OF_IMAGES, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

dataset = datasets.ImageFolder(root=DATASET, transform=data_transforms)

class_names = dataset.classes

for i, class_name in enumerate(class_names):
    print(f"Class {i}: {class_name}")

In [ ]:
train_size = int(0.8 * len(train_dataset))
test_size = int((len(train_dataset) - train_size) * 0.5)
valid_size = len(train_dataset) - train_size - test_size

train, valid, test = torch.utils.data.random_split(train_dataset, [train_size, valid_size, test_size])

train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=valid, batch_size=32, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=32, shuffle=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Usando {device} para entrenar')

## Definición de modelo básico

In [ ]:
class FaceAuthenticityCNN(nn.Module):
    def __init__(self):
        super(FaceAuthenticityCNN, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(65536, 128)
        self.fc2 = nn.Linear(128, 2)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = x.view(-1, 64 *32 *32)

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)

        return x

## Entrenamiento y evaluación

In [ ]:
model = FaceAuthenticityCNN().to(device)

def train_and_evaluate(model, train_loader, valid_loader, criterion, optimizer, num_epochs, device):
    model.to(device)
    train_losses = []
    valid_losses = []
    train_accuracies = []
    valid_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)

            labels = labels.squeeze()  

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            _, predicted_train = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted_train == labels).sum().item()

        train_loss /= len(train_loader)
        train_accuracy = correct_train / total_train
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        model.eval()
        valid_loss = 0.0
        correct_valid = 0
        total_valid = 0

        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)

                labels = labels.squeeze()  

                loss = criterion(outputs, labels)
                valid_loss += loss.item()

                _, predicted_valid = torch.max(outputs.data, 1)
                total_valid += labels.size(0)
                correct_valid += (predicted_valid == labels).sum().item()

        valid_loss /= len(valid_loader)
        valid_accuracy = correct_valid / total_valid
        valid_losses.append(valid_loss)
        valid_accuracies.append(valid_accuracy)

        print(f'Epoch {epoch + 1}/{num_epochs}, '
              f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy * 100:.2f}%, '
              f'Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy * 100:.2f}%')

    return model

model = FaceAuthenticityCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

trained_model = train_and_evaluate(model, train_loader, valid_loader, criterion, optimizer, 10, device)

## Resultados

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epoch, train_losses, label='Training Loss')
plt.plot(epoch, valid_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epoch, train_accuracies, label='Training Accuracy')
plt.plot(epoch, valid_accuracies, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
DATASET = os.path.join('real_and_fake_face')

## Preparación de datos

In [ ]:
INITIAL_SIZE_OF_IMAGES = (256, 256)

data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(INITIAL_SIZE_OF_IMAGES, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

dataset = datasets.ImageFolder(root=DATASET, transform=data_transforms)

class_names = dataset.classes

for i, class_name in enumerate(class_names):
    print(f"Class {i}: {class_name}")

In [ ]:
train_size = int(0.8 * len(dataset))
test_size = int((len(dataset) - train_size) * 0.5)
valid_size = len(dataset) - train_size - test_size

train, valid, test = torch.utils.data.random_split(dataset, [train_size, valid_size, test_size])

train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset=valid, batch_size=32, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=32, shuffle=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Usando {device} para entrenar')

## Preparación del modelo

In [ ]:
resnet18 = models.resnet18(pretrained=True)

for param in resnet18.parameters():
    param.requires_grad = False

resnet18.fc = nn.Sequential(
    nn.Linear(resnet18.fc.in_features, 512),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.ReLU(inplace=True),
    nn.Dropout(0.3),
    nn.Linear(256, 2),
)

model = resnet18.to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.001)

## Entrenamiento

In [ ]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, epochs, device="cuda"):
    model.train()
    
    train_losses = []
    valid_losses = []
    train_accuracies = []
    valid_accuracies = []

    for epoch in range(epochs):
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted_train = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted_train == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = correct_train / total_train
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        model.eval()
        valid_loss = 0.0
        correct_valid = 0
        total_valid = 0

        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                valid_loss += loss.item()

                _, predicted_valid = torch.max(outputs.data, 1)
                total_valid += labels.size(0)
                correct_valid += (predicted_valid == labels).sum().item()

        valid_loss /= len(valid_loader)
        valid_accuracy = correct_valid / total_valid
        valid_losses.append(valid_loss)
        valid_accuracies.append(valid_accuracy)

        print(f'Epoch {epoch + 1}/{epochs}, '
              f'Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy * 100:.2f}%, '
              f'Validation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy * 100:.2f}%')

    print('Training complete.')
    return train_losses, valid_losses, train_accuracies, valid_accuracies


train_losses, valid_losses, train_accuracies, valid_accuracies = train_model(model, train_loader, valid_loader, criterion, optimizer, 10, device=device)


## Resultados

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(valid_losses, label='Valid Loss')
plt.legend()
plt.xlabel('Epochs')
plt.title('Loss vs. Epochs')

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(valid_accuracies, label='Valid Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.title('Accuracy vs. Epochs')

plt.show()